<a href="https://colab.research.google.com/github/chig/genesis_tutorial_with_python/blob/main/tutorial-3.3/GENESIS_tutorial_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GENESIS Tutorial on Google Colab.**

From "*3.3 MD simulation of Protein G in NaCl solution with the CHARMM force field*":  https://www.r-ccs.riken.jp/labs/cbrt/tutorials2022/tutorial-3-3/

and "*3.2 MD simulation of alanine-tripeptide in water with the CHARMM force field*" : https://www.r-ccs.riken.jp/labs/cbrt/tutorials2022/tutorial-3-2/


These sites are designed to be downloaded as linked tar balls and executed in your environment.

However, in this tutorial, it will be reading and analyzing output files of MD simulations that have been calculated on Fugaku beforehand.

**Today's procedure**

0.   Preparation : Prepare today's material
1.   Setup : Build initial structure of simulation system (almost skipped)
2.   Minimization : Minimize energy to remove unstable steric clashes.
3.   Equilibration : Set temperature and pressure
4.   Production : Execute MD simulations for investigation
5.   Analysis : Analyze trajectories

**0. Preparation**

Please download material from github. https://github.com/chig/genesis_tutorial_with_python.git

In [ ]:
!git clone https://github.com/chig/genesis_tutorial_with_python.git

Please check if you can download data.

In [ ]:
!ls

We also use the following python modules
1. py3Dmol
2. matplotlib

To see molecules on google colab, py3Dmol (3Dmol) 
http://3dmol.csb.pitt.edu/ is a good tool.

py3Dmol can be installed on our environment via pip.

In [ ]:
!pip install py3Dmol
import py3Dmol

We can just import matplotlib since it is prepared on google colab.

In [ ]:
import matplotlib.pyplot as plt

**Tutorial material**

In [ ]:
%cd genesis_tutorial_with_python/tutorial-3.3

You can find 5 directories for steps.

In [ ]:
!ls

**1. Setup**

Setup is an improtant procedure, however, it is not today's focus.
To find the information, please access the original pages;

https://www.r-ccs.riken.jp/labs/cbrt/tutorials2022/tutorial-3-3/#1_Setup

https://www.r-ccs.riken.jp/labs/cbrt/tutorials2022/tutorial-2-2/

You can see the initial structure using py3Dmol.

In [ ]:
%cd 1_setup/

In [ ]:
with open("5_ionize/ionized.pdb") as imol:
    system = "".join([x for x in imol])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'resn' : 'SOD'}, {'sphere': {'color': 'yellow'}})
view.setStyle({'resn': 'CLA'}, {'sphere': {'color': 'green'}})
view.setStyle({'chain': 'P'},{'cartoon': {'color': 'blue'}})
view.setStyle({'atom': ['OH2']},{'sphere': {'color': 'white', 'opacity': 0.7}})
view.zoomTo()
view.show()

**2. Minimization**

First, we need to execute energy minimization to stabilize the structure.

In [ ]:
%cd ../2_minimize
!ls

'INP' is an input control of GENESIS.

In [ ]:
!cat INP

A python script (get_genesis_log/genesislog.py) is prepared to extract values from a GENESIS log file.  Please import the script.

In [ ]:
import sys
sys.path.append("../../get_genesis_log")
import genesislog

We can confirm the potential energy is decreased during the minimization.

In [ ]:
(x,y)=genesislog.read_genesis("log", "POTENTIAL_ENE")
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(x,y)
ax.set_xlabel("Steps", size=14)
ax.set_ylabel("Potential Energy (kcal/mol)",size=14)
plt.show()

**3. Equilibration**

We run three simulations at equilibration step.


1.   NVT ensemble with positional restraints on protein heavy atoms
2.   NPT ensemble with positional restraints on protein heavy atoms
3.   NPT ensemble with positional restraints on protein backbone heavy atoms


In [ ]:
%cd ../3_equilibrate
!ls

NVT ensemble with positional restraints on protein heavy atoms

In [ ]:
!cat INP1

In [ ]:
(x,y)=genesislog.read_genesis("log1", "TEMPERATURE")
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(x,y)
ax.set_xlabel("Steps", size=14)
ax.set_ylabel("Temperature (K)",size=14)
plt.show()

NPT ensemble with positional restraints on protein heavy atoms

In [ ]:
!cat INP2

In [ ]:
(x,y)=genesislog.read_genesis("log2", "BOXX")
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(x,y)
ax.set_xlabel("Steps", size=14)
ax.set_ylabel("BOXX (Angstrom)",size=14)
plt.show()

NPT ensemble with positional restraints on protein backbone heavy atoms

In [ ]:
!cat INP3

In [ ]:
(x,y)=genesislog.read_genesis("log3", "BOXX")
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(x,y)
ax.set_xlabel("Steps", size=14)
ax.set_ylabel("BOXX (Angstrom)",size=14)
plt.show()

**4. Production**

Since we need to execute very long simulations at the production step, we need to run simulations with different submissions.

In [ ]:
%cd ../4_production/
!ls

In [ ]:
!cat INP1

After the simulations, water molecules can be spread out of the simulation box and protein can be translated and rotated.

In [ ]:
with open("production_last.pdb") as imol:
    system = "".join([x for x in imol])
view = py3Dmol.view(width=400, height=300)
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'resn' : 'SOD'}, {'sphere': {'color': 'yellow'}})
view.setStyle({'resn': 'CLA'}, {'sphere': {'color': 'green'}})
view.setStyle({'chain': 'P'},{'cartoon': {'color': 'blue'}})
#view.setStyle({'atom': ['OH2', 'H1', 'H2']},{'sphere': {'color': 'white', 'opacity': 0.7}})
view.zoomTo()
view.show()

**5. Analysis**

GENESIS prepares tools to analyze trajectory files.
1. Convert trajectories to place water molecules inside the simulation box.
2. Extract a protein molecule from the trajectories
3. Calculate RMSD

In [ ]:
%cd ../5_analysis/
!ls

In [ ]:
%cd 1_crd_convert_wrap/
!ls

In [ ]:
!cat INP

In [ ]:
with open("production_last_wrap.pdb") as imol:
    system = "".join([x for x in imol])
view = py3Dmol.view(width=400, height=300)
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'resn' : 'SOD'}, {'sphere': {'color': 'yellow'}})
view.setStyle({'resn': 'CLA'}, {'sphere': {'color': 'green'}})
view.setStyle({'chain': 'P'},{'cartoon': {'color': 'blue'}})
#view.setStyle({'atom': ['OH2', 'H1', 'H2']},{'sphere': {'color': 'white', 'opacity': 0.7}})
view.zoomTo()
view.show()

In [ ]:
%cd ../2_crd_convert_pro/
!ls

In [ ]:
!cat INP

We can see that a protein molecule exists in 'production_last_protein.pdb'

In [ ]:
with open("production_last_protein.pdb") as imol:
    system = "".join([x for x in imol])
view = py3Dmol.view(width=400, height=300)
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'resn' : 'SOD'}, {'sphere': {'color': 'yellow'}})
view.setStyle({'resn': 'CLA'}, {'sphere': {'color': 'green'}})
view.setStyle({'chain': 'P'},{'cartoon': {'color': 'blue'}})
#view.setStyle({'atom': ['OH2', 'H1', 'H2']},{'sphere': {'color': 'white', 'opacity': 0.7}})
view.zoomTo()
view.show()

In [ ]:
%cd ../3_rmsd/
!ls

In [ ]:
!cat INP

In [ ]:
!head -20 output.rms

In [ ]:
x=[]
y=[]
f = open('output.rms','r')
for line in f:
  data = line[:-1].split()
  x.append(float(data[0]))
  y.append(float(data[1]))
f.close()
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(x,y)
ax.set_xlabel("Frames", size=14)
ax.set_ylabel("RMSD (Angstrom)",size=14)
plt.show()